## Script used to generate 'processed' 3d video files by smoothing (averaging) the pixel intensities across z consecutive z slices

In [1]:
## Importing the necessary modules

import numpy as np
import os
import skimage.io as io
from tqdm import tqdm
import scipy as sp
import time
import scipy.io as sio
import hdf5storage
from multiprocessing import TimeoutError
from multiprocessing.pool import ThreadPool as Pool
from functools import partial
import matplotlib.pyplot as plt
from aicspylibczi import CziFile
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Load Data

# Folder address where the raw 3D movie is
data_folder = 'E:\\Spandan\\3D_Lattice_Lightsheet\\Shen 1-30-23\\dicty_factin_pip3-06_processed'
czi_path = data_folder +'\\dicty_factin_pip3-06_processed.czi'

input_img = CziFile(czi_path)
shape = input_img.size; dims = input_img.dims;  # Check shape of the data
print(shape), print(dims)

num_timepoints, num_stacks, height, width = shape[1], shape[3], shape[4], shape[5]

(1, 241, 2, 114, 2048, 475)
STCZYX


In [3]:
def smoothing(k, img):
    
    def pre_processing(frm_id, image, ch_id):
        img_temp = image.read_image(T=frm_id, C=ch_id)
        imge = np.squeeze(img_temp[0])
        return imge

    current_frame = pre_processing(k, img, channel)

    if k == start_frame or k == end_frame - jump:
        result = current_frame
    else:
        previous_frame = pre_processing(k - jump, img, channel)
        next_frame = pre_processing(k + jump, img, channel)
        
        if k == start_frame + jump or k == end_frame - 2*jump:
            result = np.average([previous_frame, current_frame, next_frame], weights=kernel1, axis=0)
        
        else:
            frame_minus_2_jump = pre_processing(k - 2 * jump, img, channel)
            frame_plus_2_jump = pre_processing(k + 2 * jump, img, channel)
            result = np.average([frame_minus_2_jump, previous_frame, current_frame, next_frame, frame_plus_2_jump], weights=kernel2, axis=0)

    result = result.astype(np.float32)
    np.save(os.path.join(data_folder, 'smoothed_frames', f'{k}'), result)

# Example usage:
jump = 1
start_frame = 0
end_frame = num_timepoints
channel = 0
# Create a local folder for storing smoothed frames
os.makedirs(os.path.join(data_folder, 'smoothed_frames'), exist_ok=True)

In [ ]:
kernel1 = np.array([0.225,0.55,0.225]);
kernel2 = np.array([0.036,0.249,0.431,0.249,0.036])

numProcessors = 16   # maybe change it to 8 (but that's it!)
pool = Pool(processes=numProcessors)
    
print('Post-processing in parallel with '+str(numProcessors)+' processors')

#store start time
stopwatchStart = time.time()
[temp] =zip(*pool.map(partial(smoothing, img = input_img), range(num_timepoints-1)))
  
print('Wall time = '+str(np.round(time.time() - stopwatchStart,2))+' s')

print('Parallel post processing complete, switching to serial')
pool.close()

Post-processing in parallel with 16 processors
